In [16]:
import spacy
import re

# Cargamos el modelo de spaCy (inglés)
nlp = spacy.load("en_core_web_sm")

# Lista de posibles palabras que queremos mantener aunque sean ADJ
# (por ejemplo, "olive" en "olive oil", "canola" en "canola oil", etc.)
ESSENTIAL_ADJECTIVES = {"olive", "canola", "coconut", "sesame", "almond"}  
# Añade más según tus necesidades


def clean_ingredient_with_pos(text: str) -> str:
    """
    Usando spaCy, tokenizamos y etiquetamos 'text'.
    - Ignoramos tokens etiquetados como adverbios (ADV), verbos (VERB), etc.
    - Mantenemos sustantivos (NOUN, PROPN).
    - Mantenemos adjetivos (ADJ) sólo si consideramos que son parte esencial del ingrediente (heurística).
    """
    text = text.replace("-", " ")
    doc = nlp(text)
    
    keep_tokens = []
    
    for token in doc:
        # Algunas reglas simples:
        
        # 1) Si es un sustantivo (NOUN o PROPN), lo mantenemos.
        if token.pos_ in ["NOUN", "PROPN"]:
            keep_tokens.append(token.text)
        
        # 2) Si es un adjetivo (ADJ), comprobamos si lo queremos conservar
        #    por ejemplo "olive" en "olive oil".
        elif token.pos_ == "ADJ":
            # Si esa palabra está en ESSENTIAL_ADJECTIVES, la guardamos
            lower_tok = token.text.lower()
            if lower_tok in ESSENTIAL_ADJECTIVES:
                keep_tokens.append(token.text)
        
        # 3) Ignoramos ADV (adverbios: “finely”) o participios (“chopped”) o “diced”, etc.
        #    También ignoramos DET (determinantes: “the”, “a”) y conjunciones.
        else:
            pass  # no hacemos nada, los ignoramos

    # Reconstruimos la secuencia final
    # A veces conviene unirlo con espacios (o guiones si hiciera falta)
    cleaned = " ".join(keep_tokens)

    # Quitamos dobles espacios, comas, etc.
    cleaned = re.sub(r"\s+", " ", cleaned).strip(",. ")
    
    return cleaned

In [18]:
clean_ingredient_with_pos("freeze-dried strawberries")

'strawberries'